In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import logging
import pickle
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial
import gc
from importlib import reload
import pyreadstat

In [2]:
sas_path = "/home/hashjamm/project_data/disease_network/sas_files/"

In [3]:
target_diseases = pyreadstat.read_sas7bdat(sas_path + "target_diseases" + ".sas7bdat")[0]
target_diseases['COUNT'] = target_diseases['COUNT'].astype(int)
target_diseases['cum_freq'] = target_diseases['cum_freq'].astype(int)

In [4]:
target_diseases

,abb_sick,COUNT,PERCENT,cum_freq,cum_percent
0,J03,96236,4.058514,96236,4.058514
1,J20,80957,3.414160,177193,7.472673
2,J06,71639,3.021196,248832,10.493870
3,J02,64745,2.730459,313577,13.224329
4,J00,64331,2.713000,377908,15.937328
...,...,...,...,...,...
1182,T37,10,0.000422,2369577,99.931006
1183,T54,10,0.000422,2369587,99.931428
1184,T75,10,0.000422,2369597,99.931849
1185,T94,10,0.000422,2369607,99.932271


In [5]:
kcd6 = pyreadstat.read_sas7bdat(f'{sas_path}/kcd6.sas7bdat', encoding='cp949')[0]
kcd6 = kcd6[kcd6['code'].str.len() == 3]
kcd6 = kcd6.drop_duplicates(subset=['code'])
kcd6.reset_index(drop=True, inplace=True)

In [6]:
kcd6

,code,Korean,English
0,A00,콜레라,Cholera
1,A01,장티푸스 및 파라티푸스,Typhoid and paratyphoid fevers
2,A02,기타 살모넬라 감염,Other salmonella infections
3,A03,시겔라증,Shigellosis
4,A04,기타 세균성 장 감염,Other bacterial intestinal infections
...,...,...,...
2044,Z95,삼장 및 혈관 삽입물 및 이식편의 존재,Presence of cardiac and vascular implants and ...
2045,Z96,기타 기능성 삽입물의 존재,Persence of other functional implants
2046,Z97,기타 장치의 존재,Presence of other devices
2047,Z98,기타 수술후 상태,Other postsurgical states


In [9]:
result = pd.merge(target_diseases, kcd6, left_on='abb_sick', right_on='code', how='left')[['code', 'COUNT', 'PERCENT', 'Korean', 'English']]

In [ ]:
result.rename(columns={'COUNT': 'count', 'PERCENT': 'percent', 'Korean': 'korean', 'English': 'english'}, inplace=True)

In [12]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187 entries, 0 to 1186
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   code     1187 non-null   object 
 1   count    1187 non-null   int64  
 2   percent  1187 non-null   float64
 3   korean   1187 non-null   object 
 4   english  1187 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 46.5+ KB


In [16]:
result = result.astype({'code': 'string', 'count': 'int64', 'percent': 'float64', 'korean': 'string', 'english': 'string'})

In [20]:
result.to_csv('/home/hashjamm/project_data/disease_network/target_diseases_info.csv', index=False, encoding='utf-8-sig')